In [21]:
import sklearn
print(sklearn.__version__)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

1.6.1


## Read Dataset

In [22]:
preprocessed_df = pd.read_pickle("idealize_1.pkl")

##  Train Test Split

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
x = preprocessed_df.drop(columns=["RainTomorrow"])
y = preprocessed_df["RainTomorrow"]


In [25]:
preprocessed_df.shape

(113754, 20)

In [26]:
preprocessed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113754 entries, 0 to 113753
Data columns (total 20 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Location       113754 non-null  float64
 1   MinTemp        113754 non-null  float64
 2   MaxTemp        113754 non-null  float64
 3   Rainfall       113754 non-null  float64
 4   WindGustDir    113754 non-null  float64
 5   WindGustSpeed  113754 non-null  float64
 6   WindDir9am     113754 non-null  float64
 7   WindDir3pm     113754 non-null  float64
 8   WindSpeed9am   113754 non-null  float64
 9   WindSpeed3pm   113754 non-null  float64
 10  Humidity9am    113754 non-null  float64
 11  Humidity3pm    113754 non-null  float64
 12  Pressure9am    113754 non-null  float64
 13  Pressure3pm    113754 non-null  float64
 14  Temp9am        113754 non-null  float64
 15  Temp3pm        113754 non-null  float64
 16  RainTomorrow   113754 non-null  int64  
 17  Year           113754 non-nul

In [27]:
len(X_train), len(X_test), len(y_train), len(y_test)

(91003, 22751, 91003, 22751)

## Try DL Models

In [28]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

# Assuming your target column is named 'target'
# Neural Network Model
model = Sequential([
    Input(shape=(x.shape[1],)),   # automatically uses 19 input features
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(x,y, epochs=20, batch_size=32,
                    validation_split=0.2, verbose=1)

# Save the trained model
model.save("nn_model.h5")
print("✅ Neural Network model saved as nn_model.h5")


# # Evaluate the model
# loss, accuracy = model.evaluate(X_test, y_test)
# print(f"Test Accuracy: {accuracy:.4f}")
print("done")

Epoch 1/20
2844/2844 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.8098 - loss: 0.4238 - val_accuracy: 0.8492 - val_loss: 0.3543
Epoch 2/20
2844/2844 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8435 - loss: 0.3685 - val_accuracy: 0.8504 - val_loss: 0.3497
Epoch 3/20
2844/2844 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8420 - loss: 0.3683 - val_accuracy: 0.8498 - val_loss: 0.3495
Epoch 4/20
2844/2844 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8444 - loss: 0.3660 - val_accuracy: 0.8508 - val_loss: 0.3473
Epoch 5/20
2844/2844 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - accuracy: 0.8453 - loss: 0.3635 - val_accuracy: 0.8523 - val_loss: 0.3457
Epoch 6/20
2844/2844 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - accuracy: 0.8448 - loss: 0.3652 - val_accuracy: 0.8522 - val_loss: 0.3450
Epoch 7/20
2844/2844 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - accuracy: 0.8442 - loss: 0.3632 - val_accuracy: 0.8515 - val_loss: 0.3443
Epoch 8/20
2844/2844 ━━━━━━━━━━━━━━━━━━━━ 28s 8ms/step - accuracy: 0.8463 - loss: 0.358

✅ Neural Network model saved as nn_model.h5
done


In [29]:
import pandas as pd
from tensorflow.keras.models import load_model

# Load the test data
df_test = pd.read_pickle("idealize_1_test.pkl")

# Save the 'id' column to merge later
ids = df_test['id']

# Drop unnecessary columns
X_final_test = df_test.drop(['id','Day'], axis=1)

# Load the trained neural network model
model = load_model("nn_model.h5")

# Ensure input shape matches the training features (19 features expected)
assert X_final_test.shape[1] == 19, "Input data must have 19 features!"

# Predict probabilities
pred_probs = model.predict(X_final_test)

# Convert probabilities to class labels (0 or 1)
predictions = (pred_probs > 0.5).astype(int).flatten()

# Create submission DataFrame
submission = pd.DataFrame({
    'id': ids,
    'RainTomorrow': predictions
})

# Save to Excel
submission.to_csv("submission_nn.csv", index=False)
print("✅ Neural Network predictions saved to submission_nn.csv")


889/889 ━━━━━━━━━━━━━━━━━━━━ 1s 621us/step
✅ Neural Network predictions saved to submission_nn.csv


In [30]:
import pickle

best_acc = 0
best_model = None
best_name = ""

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    print(f"{name}: Accuracy = {acc:.4f}")
    
    if acc > best_acc:
        best_acc = acc
        best_model = model
        best_name = name

print(f"\nBest model: {best_name} with accuracy {best_acc:.4f}")

# Save the best model to a pickle file
with open("best_model.pkl", "wb") as f:
    pickle.dump(best_model, f)

print("Best model saved as 'best_model.pkl'")


NameError: name 'models' is not defined

In [ ]:
# Ensure RainTomorrow is categorical (0/1)
if preprocessed_df['RainTomorrow'].dtype == float:
    preprocessed_df['RainTomorrow'] = preprocessed_df['RainTomorrow'].round().astype(int)
elif preprocessed_df['RainTomorrow'].dtype == object:
    preprocessed_df['RainTomorrow'] = preprocessed_df['RainTomorrow'].map({'No': 0, 'Yes': 1})